In [3]:
#
# SCRIPT 1: preprocess.py
#
# PURPOSE:
# This script prepares the data for datahub
# It loads the original, large '.npz' file (which contains ALL data) into RAM one time.
# It then splits the data into training, validation, and testing sets.
# Finally, it saves each of these sets as its *own* separate '.npy' file.
# This allows our main 'train.py' script to load only the small batches it needs,
# preventing the Out-of-Memory (OOM) errors.
#

import numpy as np
from sklearn.model_selection import train_test_split
import os

# --- 1. DEFINE FILE PATHS ---
# Define the directory where the data is stored.
DATA_DIR = "./" 

# This is the INPUT file downloaded from the authors.
npz_file = os.path.join(DATA_DIR, 'RWs_H_g_2p2_tadv_1min_Hs_g_1m.npz')

# These are the OUTPUT files we are creating.
x_train_file = os.path.join(DATA_DIR, 'x_train.npy')
y_train_file = os.path.join(DATA_DIR, 'y_train.npy')
x_val_file = os.path.join(DATA_DIR, 'x_val.npy')
y_val_file = os.path.join(DATA_DIR, 'y_val.npy')
x_test_file = os.path.join(DATA_DIR, 'x_test.npy')
y_test_file = os.path.join(DATA_DIR, 'y_test.npy')

# --- 2. DATA PREPROCESSING ---
print(f"Loading {npz_file}...")
try:
    # Load the original .npz file into the computer's main RAM.
    with np.load(npz_file) as data:
        # The .npz file is like a zip file for arrays. We extract the ones we need.
        # This 'wave_data_train' array is actually 80% of the total data.
        wave_data_train_full = data['wave_data_train']
        label_train_full = data['label_train']
        # This is the final 20% test set.
        wave_data_test = data['wave_data_test']
        label_test = data['label_test']

    print("Data loaded. Splitting training and validation data...")
    
    # We split the 'wave_data_train_full' (80% of total) into a new, smaller
    # training set (64%) and a validation set (16%).
    # This matches the 64/16/20 split mentioned in the paper.
    x_train, x_val, y_train, y_val = train_test_split(
        wave_data_train_full,  # The 80% we loaded
        label_train_full, 
        test_size=0.20,      # 20% of the 80% = 16% of the total
        random_state=42      # Ensures the split is reproducible
    )

    print("Saving data to separate .npy files...")
    
    # This is the key step: save each new array as its own file.
    # The 'train.py' script will read from these.
    np.save(x_train_file, x_train)
    np.save(y_train_file, y_train)
    np.save(x_val_file, x_val)
    np.save(y_val_file, y_val)
    np.save(x_test_file, wave_data_test)
    np.save(y_test_file, label_test)
    
    print("\nPreprocessing complete. Run the 'train_on_datahub.py' script.")

except FileNotFoundError:
    print(f"ERROR: Could not find '{npz_file}'.")
    print("Please make sure it is in the same directory.")
except Exception as e:
    print(f"An error occurred: {e}")

Loading ./RWs_H_g_2p2_tadv_1min_Hs_g_1m.npz...
Data loaded. Splitting training and validation data...
Saving data to separate .npy files...

Preprocessing complete. You can now run the 'train_on_datahub.py' script.


In [4]:
#
# SCRIPT 2: train_on_datahub.py
# (Run this script after 'preprocess.py' to train the model)
#
# PURPOSE:
# This script trains our LSTM model.
# It uses a custom 'DataGenerator' class to stream data from the '.npy' files created in 'preprocess.py'. 
# This is the solution to the Datahub GPU memory limit. Instead of loading the whole dataset onto the GPU, this class feeds the model tiny batches (16 samples at a time).
#

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, BatchNormalization, Dropout, Dense, Input
from tensorflow.keras.optimizers import Adam
# Callbacks are to make training smarter
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os

# --- 1. DEFINE FILE PATHS ---
# These are the INPUT files (created by preprocess.py above)
DATA_DIR = "./" 
x_train_file = os.path.join(DATA_DIR, 'x_train.npy')
y_train_file = os.path.join(DATA_DIR, 'y_train.npy')
x_val_file = os.path.join(DATA_DIR, 'x_val.npy')
y_val_file = os.path.join(DATA_DIR, 'y_val.npy')
x_test_file = os.path.join(DATA_DIR, 'x_test.npy')
y_test_file = os.path.join(DATA_DIR, 'y_test.npy')

# This is the OUTPUT file: the single best trained model
best_model_file = os.path.join(DATA_DIR, 'best_rogue_wave_model.keras')

# --- 2. KERAS DATA GENERATOR ---
# This class is the core of solving the GPU issue.
# It inherits from 'tf.keras.utils.Sequence', which is the standard
# way to build a data pipeline for Keras.
class DataGenerator(tf.keras.utils.Sequence):
    # 'self' is the object, '__init__' is the "constructor" that sets it up
    def __init__(self, x_file, y_file, batch_size):
        # Load the .npy files into RAM. This is fine, Datahub has enough RAM.
        # We are *not* loading them onto the tiny GPU.
        self.x = np.load(x_file) 
        self.y = np.load(y_file)
        self.batch_size = batch_size
        self.num_samples = self.x.shape[0]
        self.indices = np.arange(self.num_samples)
        self.on_epoch_end() # Shuffle the data once at the start

    def __len__(self):
        # Keras needs to know how many batches are in one epoch.
        # e.g., 14927 samples / 16 batch_size = 932 batches
        return int(np.floor(self.num_samples / self.batch_size))

    def __getitem__(self, index):
        # This is the function Keras calls to get a batch of data.
        # 'index' is the batch number (e.g., batch #0, batch #1, ...)
        
        # Get the 16 indices for this specific batch
        batch_indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        
        # Get the data from RAM for just those 16 samples
        x_batch = self.x[batch_indices]
        y_batch = self.y[batch_indices]
        
        # Return the small batch, which Keras then sends to the GPU
        return x_batch, y_batch
        
    def on_epoch_end(self):
        # This function is called by Keras at the end of every epoch
        # to shuffle the data, so the model sees it in a different order.
        np.random.shuffle(self.indices)

# --- 3. MODEL TRAINING ---

# Define Model Constants from the paper (Table 2)
N_TIMESTEPS = 1536  # Each sample is 1536 data points long
N_FEATURES = 1      # We only have one feature: wave height
N_LSTM_UNITS = 10   # Number of neurons in each LSTM layer
N_LSTM_LAYERS = 4   # Number of LSTM layers to stack
DROPOUT_PROB = 0.05
LEARNING_RATE = 0.001
BATCH_SIZE = 16     # Small batch size for low GPU memory
EPOCHS = 100        # Train for a long time. EarlyStopping will find the best one.

try:
    print("Initializing Data Generators...")
    # Create one generator for each data split
    train_gen = DataGenerator(x_train_file, y_train_file, BATCH_SIZE)
    val_gen = DataGenerator(x_val_file, y_val_file, BATCH_SIZE)
    test_gen = DataGenerator(x_test_file, y_test_file, BATCH_SIZE)
    print("Data Generators created successfully.")

    # --- Build the LSTM Model (from Figure 4) ---
    model = Sequential(name="Rogue_Wave_LSTM")
    model.add(Input(shape=(N_TIMESTEPS, N_FEATURES)))
    
    # Loop to create the 4 stacked LSTM + BatchNormalization layers
    for i in range(N_LSTM_LAYERS):
        # 'return_sequences=True' is needed for all but the *last* LSTM layer
        # so it can pass its full output sequence to the next layer.
        is_last_layer = (i == N_LSTM_LAYERS - 1)
        model.add(LSTM(N_LSTM_UNITS, return_sequences=not is_last_layer))
        model.add(BatchNormalization())
    
    model.add(Dropout(DROPOUT_PROB))
    
    # Final output layer. 'Dense(2)' gives two outputs (Prob for class 0, Prob for class 1)
    # 'softmax' ensures these two probabilities add up to 1.0.
    model.add(Dense(2, activation='softmax'))
    
    model.summary() # Prints the model architecture
    
    # --- Compile the Model ---
    model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE),
        # 'sparse_categorical_crossentropy' is the correct loss function
        # when labels are integers (0, 1) and the model has >1 output neuron.
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # --- 4. DEFINE CALLBACKS ---
    
    # This callback saves the model to a file *only* when the 'val_accuracy'
    # improves. This way, we always have the "best" version saved.
    checkpoint_callback = ModelCheckpoint(
        filepath=best_model_file, # File to save to
        monitor='val_accuracy',   # Metric to watch
        save_best_only=True,      # Only save if it's the new best
        mode='max',               # 'max' because we want *max* accuracy
        verbose=1                 # Print a message when it saves
    )
    
    # This callback stops the training early if the model isn't improving.
    early_stopping_callback = EarlyStopping(
        monitor='val_accuracy',     # Metric to watch
        patience=15,              # Stop if no improvement for 15 epochs
        mode='max',
        verbose=1,
        # This is the magic: it reloads the weights from the 'best_model_file'
        # when it finishes. So, the 'model' object will be the best version.
        restore_best_weights=True 
    )

    # --- 5. TRAIN THE MODEL ---
    print("\n--- Starting Model Training (for up to 100 epochs) ---")
    
    # We pass the 'generators' to model.fit, not the raw numpy arrays.
    history = model.fit(
        train_gen,
        epochs=EPOCHS,
        validation_data=val_gen,
        workers=2, # Use 2 CPU cores to prepare data batches in parallel
        callbacks=[checkpoint_callback, early_stopping_callback] 
    )
    
    # --- 6. EVALUATE THE BEST MODEL ON THE TEST SET ---
    # Because of 'restore_best_weights=True', 'model' is already the best version.
    
    print("\n--- Evaluating Best Model on Test Set ---")
    
    # We use 'test_gen' (the final 20% of data) to get our final score.
    # 'verbose=0' stops it from flooding the console with messages.
    test_results = model.evaluate(test_gen, verbose=0) 
    
    print(f"\nTest Loss: {test_results[0]:.4f}")
    print(f"Test Accuracy: {test_results[1]:.4f}")
    
    print(f"\nPaper's reported accuracy was ~76% (0.7592).")
    if abs(test_results[1] - 0.7592) < 0.03:
        print("Success!")
    else:
        print(f"Result: {test_results[1]*100:.2f}%")

except FileNotFoundError:
    print("ERROR: .npy files not found.")
    print("Run the 'preprocess' script first")
except Exception as e:
    print(f"An error occurred during training/evaluation: {e}")

Initializing Data Generators...
Data Generators created successfully.
Model: "Rogue_Wave_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 1536, 10)          480       
                                                                 
 batch_normalization_4 (Bat  (None, 1536, 10)          40        
 chNormalization)                                                
                                                                 
 lstm_5 (LSTM)               (None, 1536, 10)          840       
                                                                 
 batch_normalization_5 (Bat  (None, 1536, 10)          40        
 chNormalization)                                                
                                                                 
 lstm_6 (LSTM)               (None, 1536, 10)          840       
                                               